In [12]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import model_from_json
import numpy as np
import os

In [13]:
batch_size = 32
num_classes = 10
epochs = 100
num_predictions = 20

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [14]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [15]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
  960/50000 [..............................] - ETA: 3:21 - loss: 2.2932 - acc: 0.1156

KeyboardInterrupt: 

In [ ]:
model_path_cifar = os.getcwd() + 'flask/static/CIFAR/models/'
filename = 'CNN_model.h5'
filename_weights = 'CNN_weights'
filename_archi = 'CNN_architecture'
model.save(model_path_cifar + filename) # save entire model
model.save_weights(model_path_cifar + filename_weights) # save model weight to the filename param in the function  
with open(model_path_cifar + filename_archi, 'w') as f: # save model architecture
    f.write(model.to_json())

In [21]:
# calculate accuracy of the models
# model loading/reconstruction from JSON file  
with open('models/CIFAR/CNN_architecture.json', 'r') as f:
    model = model_from_json(f.read())  
# load weights into the new model
model.load_weights('models/CIFAR/CNN_weights.h5')

# function to get model accuracy scores
def eval_model(model):  
    global wrong_predictions_num, corr_predictions_num,total_pred
    wrong_predictions_num = 0
    corr_predictions_num = 0

    for n in range(x_test.shape[0]):
        # get true label values
        label_true = y_test[n].argmax()

        # get_prediction
#         predictions = model.predict(np.reshape(x_test[n],[1,32,32,1]))
        predictions = model.predict(np.reshape(x_test[n],[1,32,32,3]))
        predictions_lst = predictions.tolist()[0]
        max_value = max(predictions_lst)
        predicted = predictions_lst.index(max_value)

        # compute percentages
        if label_true != predicted:
            wrong_predictions_num += 1
        elif label_true == label_true:
            corr_predictions_num += 1
    
    # print results
    total_pred = corr_predictions_num+wrong_predictions_num # total number of predictions
    print('number of correct predictions: ', corr_predictions_num)
    print('number of incorrect predictions: ', wrong_predictions_num)
    print('total: ', total_pred)
    print('Accuracy: ', round(corr_predictions_num/total_pred*100,2),'%')
    
# get model accuracy scores 
print('Model Accuracy CIFAR \n')
eval_model(model)

Model Accuracy CIFAR 

number of correct predictions:  7553
number of incorrect predictions:  2447
total:  10000
Accuracy:  75.53 %
